In [ ]:
import random

import numpy as np
from enum import Enum

Methods = Enum('Methods', ['Classic', 'Momentum', 'AdaGrad', 'RMSprop', 'Adam', 'Nesterov'])
Regularization = Enum('Regularization', ['WithoutRegularization', 'L1', 'L2', 'Elastic'])
LearningRate = Enum('LearningRate', ['Const'])
LearningRateScheduling = Enum('LearningRateScheduling', ['Classic', 'Stepwise', 'Exponential'])


def sign(x):
    if x > 0:
        return 1
    elif x == 0:
        return 0
    else:
        return -1


class LinearRegression:
    def __init__(self, T, W, X, Y, regularization=Regularization.WithoutRegularization, l1=0.1, l2=0.1):
        self.T = np.array([T[i % len(T)](X[i // len(T)]) for i in range(len(T) * len(X))]).reshape(len(X), len(T))
        self.W = W
        self.X = X
        self.Y = Y
        self.regularization = regularization
        self.l1 = l1
        self.l2 = l2
        self.W_points = [np.copy(self.W)]
        self.loss_values = [self.loss(self.W)]

    def loss(self, W_Arg, is_avarage=False):
        val = sum([(np.dot(self.T[i], W_Arg) - self.Y[i]) ** 2 for i in range(len(self.X))])
        match self.regularization:
            case Regularization.L1:
                val += self.l1 * sum([abs(w) for w in self.W]) / len(self.W)
            case Regularization.L2:
                val += self.l2 * sum([w ** 2 for w in self.W]) / len(self.W)
            case Regularization.Elastic:
                val += (self.l1 * sum([abs(w) for w in self.W])) / len(self.W) + (
                        self.l2 * sum([w ** 2 for w in self.W])) / len(self.W)

        return val / len(self.X) if is_avarage else val

    def grad_by_components(self, index_components, W_Arg):
        grad_with_batch = np.zeros(len(W_Arg))
        for i in index_components:
            grad_with_batch += (2 * (np.dot(self.T[i], W_Arg) - self.Y[i]) * self.T[i])
        match self.regularization:
            case Regularization.L1:
                grad_with_batch += self.l1 * np.array([sign(w) for w in self.W]) / len(self.W)
            case Regularization.L2:
                grad_with_batch += self.l2 * 2 * self.W / len(self.W)
            case Regularization.Elastic:
                grad_with_batch += (self.l1 * np.array([sign(w) for w in self.W])) / len(self.W) + (
                        self.l2 * 2 * self.W) / len(self.W)

        return grad_with_batch

    def analytical_solution(self):
        return (np.linalg.inv(np.transpose(self.T) @ self.T) @ np.transpose(self.T)) @ self.Y


def sgd(lin_reg, lr, lrs, eps, batch, max_num_of_step, beta_1, beta_2, eps_adam, is_corr_beta_1=True,
        is_corr_beta_2=True, is_nesterov=False, is_adagrad=False, without_squares=False,
        store_points=False):
    i = 0
    prev_W = lin_reg.loss(lin_reg.W)
    V = np.zeros(len(lin_reg.W))
    S = np.zeros(len(lin_reg.W))
    lrs_func = lrs_handler(lrs)

    while True:
        i += 1

        components = [(i * batch + j) % len(lin_reg.X) for j in range(batch)]
        cur_w = lin_reg.W
        grad_with_batch = lin_reg.grad_by_components(components, cur_w)

        alpha = lrs_func(lr(lambda a: lin_reg.loss(lin_reg.W - a * grad_with_batch)), (i * batch) // len(lin_reg.X))
        if is_nesterov:
            cur_w -= alpha * beta_1 * V
            grad_with_batch = lin_reg.grad_by_components(components, cur_w)

        V = (beta_1 * V) + (1 - beta_1) * grad_with_batch
        S = (beta_2 * S) + (1 - beta_2) * (grad_with_batch ** 2) if ~is_adagrad else (S + (grad_with_batch ** 2))
        V_norm = V / (1 - (beta_1 ** (i + 1))) if is_corr_beta_1 else V
        S_norm = S / (1 - (beta_2 ** (i + 1))) if is_corr_beta_2 else S

        if without_squares:
            lin_reg.W = lin_reg.W - alpha * V_norm
        else:
            lin_reg.W = lin_reg.W - alpha * (V_norm / (((S_norm) + eps_adam) ** 0.5))

        if store_points:
            lin_reg.W_points.append(np.copy(lin_reg.W))
        lin_reg.loss_values.append(lin_reg.loss(lin_reg.W))
        if abs(lin_reg.loss(lin_reg.W) - prev_W) < eps or i >= max_num_of_step:
            break
        prev_W = lin_reg.loss(lin_reg.W)

    return i


def sgd_handler(lin_reg, lr, method, lrs=LearningRateScheduling.Classic, batch=1, beta_1=0.9, beta_2=0.999,
                eps_adam=10 ** -8,
                eps=0.001, max_num_of_step=10000, store_points=False):
    match method:
        case Methods.Classic:
            return sgd(lin_reg, lr, lrs, eps, batch, max_num_of_step, beta_1=0, beta_2=1, eps_adam=1,
                       is_corr_beta_1=False, is_corr_beta_2=False, without_squares=True, store_points=store_points)
        case Methods.Momentum:
            return sgd(lin_reg, lr, lrs, eps, batch, max_num_of_step, beta_1, beta_2=1, eps_adam=1,
                       is_corr_beta_1=False, is_corr_beta_2=False, without_squares=True, store_points=store_points)
        case Methods.AdaGrad:
            return sgd(lin_reg, lr, lrs, eps, batch, max_num_of_step, beta_1=0, beta_2=0.5, eps_adam=eps_adam,
                       is_corr_beta_1=False, is_corr_beta_2=False, is_adagrad=True, store_points=store_points)
        case Methods.RMSprop:
            return sgd(lin_reg, lr, lrs, eps, batch, max_num_of_step, beta_1=0, beta_2=beta_2, eps_adam=eps_adam,
                       is_corr_beta_1=False, store_points=store_points)
        case Methods.Adam:
            return sgd(lin_reg, lr, lrs, eps, batch, max_num_of_step, beta_1, beta_2, eps_adam,
                       store_points=store_points)
        case Methods.Nesterov:
            return sgd(lin_reg, lr, lrs, eps, batch, max_num_of_step, beta_1, beta_2=1, eps_adam=1,
                       is_corr_beta_1=False, is_corr_beta_2=False, is_nesterov=True, without_squares=True,
                       store_points=store_points)


def lrs_exp(decay, epoch_update):
    return lambda lr, t: lr / (decay ** (t // epoch_update))


def lrs_step(decay, epoch_update):
    return lambda lr, t: lr - decay * (t // epoch_update)


def lrs_handler(lrs, epoch_update=10):
    match lrs:
        case LearningRateScheduling.Classic:
            return lambda lr, t: lr
        case LearningRateScheduling.Stepwise:
            return lrs_step(0.001, epoch_update)
        case LearningRateScheduling.Exponential:
            return lrs_exp(2, epoch_update)



In [ ]:
import matplotlib.pyplot as plt


def visualise_points(lin_reg):
    x = np.linspace(min(lin_reg.X), max(lin_reg.X), 1000)
    y = sum(
        [lin_reg.W[i] * (x ** i) for i in range(len(lin_reg.W))]
    )
    analys_w = lin_reg.analytical_solution()
    analys_y = sum(
        [analys_w[i] * (x ** i) for i in range(len(analys_w))]
    )
    plt.plot(x, y, '-r')
    plt.plot(x, analys_y, '-b')
    plt.plot(lin_reg.X, lin_reg.Y, 'og', linestyle='None')
    plt.xlabel("x")
    plt.show()


def visualise_linear_sgd(lin_reg):
    values = np.reshape(lin_reg.W_points, (2, len(lin_reg.W_points)))
    X = np.linspace(min(values[0]) / 2, max(values[0]) * 2, 100)
    Y = np.linspace(min(values[1]) / 2, max(values[1]) * 2, 100)
    Z = [[lin_reg.loss(np.array((X[i], Y[j]))) for i in range(len(X))] for j in range(len(Y))]
    plt.contour(X, Y, Z, 40)

    plt.plot(values[0], values[1], marker='.')
    plt.plot(values[0][0], values[1][0], 'og')
    plt.plot(values[0][-1], values[1][-1], 'or')
    plt.xlabel('w_1')
    plt.ylabel('w_2')
    plt.show()



In [ ]:
import time
import tracemalloc


def poly_array(coeffs):
    return [lambda x, i=i: coeffs[i] * (x ** i) for i in range(len(coeffs))]


def poly(poly_arr):
    return lambda x: sum([poly_arr[i](x) for i in range(len(poly_arr))])


def generate_data(num_of_points, dimension, coeffs_left, coeffs_right, x_left, x_right, deviation):
    coeffs = np.array([float(random.randint(coeffs_left, coeffs_right)) for i in range(dimension + 1)])

    X = [random.uniform(x_left, x_right) for _ in range(num_of_points)]
    Y = [poly(poly_array(coeffs))(X[i]) + random.uniform(-deviation, +deviation) for i in range(num_of_points)]

    return [np.array(X), np.array(Y), coeffs]


def gen_linear_reg(dimension, num_of_points, coeffs_left, coeffs_right, x_left, x_right, deviation):
    T = np.array(poly_array(np.ones(dimension + 1)))
    X, Y, coeffs = generate_data(num_of_points, dimension, coeffs_left, coeffs_right, x_left, x_right, deviation)
    W = np.ones(len(coeffs))

    return LinearRegression(T, W, X, Y)


def test_universal(lin_reg, lr, method, lrs, batch=1):
    res_univ = {
        'mem': 0,
        'steps': 0,
        'time': 0,
        'sqrt': 0
    }

    start = time.time()
    tracemalloc.start()
    steps = sgd_handler(lin_reg, lr, method, lrs=lrs, batch=batch)
    res_univ['mem'] = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    end = time.time()

    res_univ['steps'] = (steps)
    res_univ['time'] = end - start
    res_univ['sqrt'] = lin_reg.loss(lin_reg.W, is_avarage=True)

    return res_univ


test_results = {}


def tests():
    for dimension in range(1, 11):
        for num_of_points in range(10, 100, 10):
            for deviation in [0. + i for i in range(6)]:
                lin_reg = gen_linear_reg(dimension=dimension, num_of_points=num_of_points, coeffs_left=-3,
                                         coeffs_right=2, x_left=1., x_right=2., deviation=deviation)
                lin_reg_info = 'Linear Regression Info: \n Dimension: {} | Number of Points: {} | Deviation: {}' \
                    .format(dimension, num_of_points, deviation)
                for method in Methods:
                    for regularization in Regularization:
                        for lr in LearningRate:
                            for lrs in LearningRateScheduling:
                                for batch in range(1, len(lin_reg.W) + 1):
                                    for lr0 in [0.1, 0.01, 0.001]:
                                        cur_lr = lambda x: lr0
                                        sgd_info = 'SGD Info: \n Batch: {} | Method: {} | Regularization: {} | Learning Rate(lr0={}): {} | Learning Rate Scheduling: {}'\
                                            .format(batch, method.name, regularization.name, lr0, lr.name, lrs.name)

                                        lin_reg.W = np.ones(len(lin_reg.W))
                                        lin_reg.W_points = [np.copy(lin_reg.W)]
                                        lin_reg.loss_values = [lin_reg.loss(lin_reg.W)]
                                        results = test_universal(lin_reg, cur_lr, method, lrs, batch)

                                        print(lin_reg_info)
                                        print(sgd_info)
                                        print('Metrics: \n ' + str(results))
                                        visualise_points(lin_reg)
                                    # visualise_linear_sgd(lin_reg) # only if dimension == 1
                                    # test_results[info] = results


tests()

for key in test_results:
    print(str(key) + " -> ")
    print('\n'.join(str(val) for val in test_results[key]))
    print()